In [1]:
import pandas as pd

## Plan

### Here are the steps you need to complete to make this function:
1. open the .yaml file with tree properties
    * use these tree properties to assign a death year for each tree based on (1) species and (2) age/year planted
    
2. develop the architecture of the .yaml file with strategies. What will this look like? What information will it hold? How flexible/rigid will it be (at first)?

3. first off, based on the initial data, determine the death year of all of the trees. append this to a column 'death year' in the new dataframe

4. start by making a function that simply creates the new event dataframe assuming no strategies are implemented. I.e. all the rows will have status of either "plant" or "replant." End year is determined by the death year, and replant is simply determined by the death year + 1.

4. add to that function the ability to add pruning, so the status will now be "plant," "replant," or "prune"
    * pruning will have to be implemented with some sort of year crieteria (in what year, or even better, at what age, should farmers prune?)
    * with the new row that represents the pruning event, death year carries over but is altered slightly (because we are working under the assumption that pruning increases production life)
    * unless another event is called, end year is death year
    
5. add to that function the ability to add intercropping.
    * intercropping will be implemented with (1) a year criteria and (2) a proportion criteria (what proportion of the trees will be replanted). 
    * if the trees in the originaal dataset reach the year or age criteria, the original row will become inactive because the 'end year' has been reached. The row will then be split into two rows (but both still show the same ID number, which is how we identify them as the same plot). The first row will be the trees that were left alone, and so it will be the exact same except that is has proportionally less cuerdas (the death year will carry over from the original). The second row will have the trees which were replanted. Their


In [2]:
data = pd.read_csv("data/demoData.csv")

In [8]:
# for now, 'Unnamed: 0' will be our ID number
yearPlanted = 2020 - data['ageOfTrees']
data['yearPlanted'] = yearPlanted
data.head()

,Unnamed: 0,farmerName,treeType,numCuerdas,ageOfTrees,yearPlanted
0,0,Estefanía Cazares,catuai,9,9,2011
1,1,Aldo Linares,catuai,1,2,2018
2,2,Aldo Linares,borbon,1,20,2000
3,3,Linda Gabriel Portillo Herrera,catuai,4,7,2013
4,4,Linda Gabriel Portillo Herrera,borbon,10,27,1993


In [ ]:
# lets say that everyone prunes when their trees hit 10 years old, to start.

for i in range